# Import

In [ ]:
import os
import time
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, BatchNormalization, Conv2D, Conv2DTranspose, ReLU, LeakyReLU, Flatten, MaxPooling2D, Dropout, Reshape

# GPU Initialization

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('DEVICES AVAILABLE: {}'.format(strategy.num_replicas_in_sync))

# Initialization

In [ ]:
#initially I went for 128*128, later decided to go with 512*512 image size
BUFFER_SIZE = 64000
BATCH_SIZE = 32*strategy.num_replicas_in_sync
batch_size = BATCH_SIZE
EPOCHS = 50
latent_dim = 128
input_size = [256*2, 256*2, 3]
image_size = (256*2, 256*2)

# Data Preprocessing

In [ ]:
#Creating dataset with ImageDataGenerator is way simpler than other methods
datagen = ImageDataGenerator(
    rescale=1./255,
# Initially created the datatest with some augmentation, then realised, bad idea.
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)

image_directory = 'chest-xray-pneumonia/chest_xray'

dataset= datagen.flow_from_directory(
    os.path.join(image_directory, 'train'),
    classes=['PNEUMONIA'],
    target_size=image_size,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)


In [ ]:
len(dataset)

# Generator Model

In [ ]:
def gen_model():
    #in case you get OOM error, change the filter size, set it to a smaller value, 28 for example
    model = Sequential([
        Input(shape = (latent_dim,)),
        Dense(8*8*256),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128*2, kernel_size = 4, strides = 2, padding = 'same'),
        LeakyReLU(alpha=0.1),
        Conv2DTranspose(128*3, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.1),
        Conv2DTranspose(128*3, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.1),
        Conv2DTranspose(128*4, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.1),
        Conv2DTranspose(128*5, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.1),
        Conv2DTranspose(128*6, kernel_size=4, strides=2, padding='same'),
        LeakyReLU(alpha=0.1),
        Conv2D(3, kernel_size =4, padding = 'same', activation = 'sigmoid')
    ],
        name = "generator"
    )
    return model

# Discriminator Model

In [ ]:
def disc_model():
    #in case you get OOM error, change the filter size, set it to a smaller value, 256 or lower for example
    #keep reducing that value untill the error goes away.
    model = Sequential([
        Input(shape = input_size),
        Conv2D(256, kernel_size = 4, strides= 2, padding = 'same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(strides = 2),
        Conv2D(256*2, kernel_size=4, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(strides=2),
        Conv2D(256*3, kernel_size=4, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(strides=2),
        Conv2D(256*4, kernel_size=4, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(strides=2),

        Flatten(),
        Dense(256*4),
        LeakyReLU(alpha=0.1),
        Dropout(0.2),
        Dense(1, activation = 'sigmoid')
    ],
        name = "discriminator"
    )
    return model



In [ ]:
with strategy.scope():
    #In order to utilize mutliple GPU,
    #you  must declare model, optimizers and checkpoints inside of a scope
    generator = gen_model()
    discriminator = disc_model()


In [ ]:
generator.summary()
discriminator.summary()

In [ ]:
#helper funtion to help us with loadidng images in batches
def image_loader(generator):
    for images, labels in generator:
        yield images, labels

# GAN with Custom Traning Step

In [ ]:
#gan model with custom gradient calculation
class Gan(Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, disc_opt, gen_opt, loss_function):
        super().compile()
        self.disc_opt = disc_opt
        self.gen_opt = gen_opt
        self.loss_function = loss_function
        self.disc_loss_metric = tf.keras.metrics.Mean(name = "disc_loss")
        self.gen_loss_metric = tf.keras.metrics.Mean(name = "gen_loss")

    @property
    def metrics(self):
        return [self.disc_loss_metric, self.gen_loss_metric]

    #custom training step
    def train_step(self, data):  # Modify the function to accept labels separately
        real_images, real_labels = data
        batch_size = tf.shape(real_images)[0]

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Fake image decoding
        generated_images = self.generator(random_latent_vectors)

        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Concatenate the real and fake labels
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )


        labels += 0.05*tf.random.uniform(tf.shape(labels))


        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            disc_loss = self.loss_function(labels, predictions)

        grads  = tape.gradient(disc_loss, self.discriminator.trainable_weights)
        self.disc_opt.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )


        random_latent_vectors = tf.random.normal(shape = (batch_size,self.latent_dim))

        misleading_labels = tf.zeros((batch_size, 1))


        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))

            gen_loss = self.loss_function(misleading_labels, predictions)

        grads = tape.gradient(gen_loss, self.generator.trainable_weights)
        self.gen_opt.apply_gradients(zip(grads, self.generator.trainable_weights))

        self.disc_loss_metric.update_state(disc_loss)
        self.gen_loss_metric.update_state(gen_loss)
        return{
            "disc_loss": self.disc_loss_metric.result(),
            "gen_loss": self.gen_loss_metric.result()
        }


In [ ]:
#helper functiont to generated a image using current generator model
#It was initially desgined to use inside of a  callback, but can be used outside of that too.
#Use 1 as defalut parameter when calling independently.
def gen_images(current_epoch):
    noise = tf.random.normal([2, latent_dim])
    num_of_sample = 2
    generated_images = generator(noise, training = False)
    figure = plt.figure(figsize=(20,20))
    for i in range(generated_images.shape[0]):
        plt.subplot(2, 2,i+1)
        plt.imshow(generated_images[i, :, :, 0, ], cmap = 'gray')
        plt.title(f"After epoch {current_epoch}")
        plt.axis('off')
    plt.savefig('After epochs{:04d}.png'.format(current_epoch))
    plt.show()

# Callbacks

In [ ]:
#callbacks. We are showing progress of gan and also saving samples after each epochs
class Gan_Callback(tf.keras.callbacks.Callback):
    def __init__(self, num_images=2, latent_dim = 128):
        self.num_images = num_images
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs =None):
        latent_vectors = tf.random.normal(shape = (self.num_images, latent_dim))
        generated_images = self.model.generator(latent_vectors)
        generated_images *=255
        generated_images.numpy()
        figure = plt.figure(figsize=(10,10))
        for i in range(generated_images.shape[0]):
            plt.subplot(2, 2,i+1)
            plt.imshow(generated_images[i, :, :, 0, ], cmap='gray')
            plt.title(f"After epoch {epoch+1}")
            plt.axis('off')
        plt.savefig('After epochs{:04d}.png'.format(epoch+1))
        plt.show()
        if(epoch % 10 ==0):
            self.model.generator.save('gen.h5')
            self.model.discriminator.save('disc.h5')

In [ ]:
with strategy.scope():
    #In order to utilize mutliple GPU,
    #you  must declare model, optimizers and checkpoints inside of a scope
    gan = Gan(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
    gan.compile(
        disc_opt=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
        gen_opt=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
        # Parallel gpu computing won't work unless  we pass reduction=tf.keras.losses.Reduction.NONE as a parameter too.
        loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE),
    )


# Traning

In [ ]:
#actual traing begins here
history = gan.fit(
    image_loader(dataset),
    epochs=EPOCHS,
    steps_per_epoch=len(dataset),
    callbacks=[Gan_Callback(num_images=4, latent_dim=latent_dim)]
)


# Model Evaluation

Generates Samples and Visual Inspections

In [ ]:
def generate_and_save_images(model, epoch, latent_dim, num_examples=4):
    random_latent_vectors = tf.random.normal(shape=(num_examples, latent_dim))
    generated_images = model.generator(random_latent_vectors)
    generated_images *= 255
    generated_images = generated_images.numpy()

    plt.figure(figsize=(10, 10))
    for i in range(num_examples):
        plt.subplot(2, 2, i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.title(f"Generated Image {i+1}")
        plt.axis('off')
    plt.savefig(f'generated_image_epoch_{epoch+1}.png')
    plt.show()

# Example usage after training
generate_and_save_images(gan, EPOCHS, latent_dim)

Quantitative Metrics

In [ ]:
from scipy.linalg import sqrtm
from numpy import cov, trace, iscomplexobj

def calculate_fid(real_images, generated_images):
    # Calculate mean and covariance statistics
    mu1, sigma1 = real_images.mean(axis=0), cov(real_images, rowvar=False)
    mu2, sigma2 = generated_images.mean(axis=0), cov(generated_images, rowvar=False)
    # Calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # Calculate sqrt of product between covariances
    covmean = sqrtm(sigma1.dot(sigma2))
    # Check and correct imaginary numbers from sqrtm
    if iscomplexobj(covmean):
        covmean = covmean.real
    # Calculate FID
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# Example usage
# real_images and generated_images should be numpy arrays of shape (num_samples, height, width, channels)
fid_score = calculate_fid(real_images, generated_images)
print(f'FID Score: {fid_score}')

# Fine-Tuning

In [ ]:
# Example of changing learning rates for fine-tuning
gan.compile(
    disc_opt=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5),
    gen_opt=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5),
    loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
)

# Continue training with new hyperparameters
history = gan.fit(
    image_loader(dataset),
    epochs=EPOCHS,
    steps_per_epoch=len(dataset),
    callbacks=[Gan_Callback(num_images=4, latent_dim=latent_dim)]
)

# Save and Export the model

Save model Weighs

In [ ]:
# Save the generator and discriminator models
generator.save('generator_model.h5')
discriminator.save('discriminator_model.h5')

Export model

In [ ]:
# Save the entire GAN model
gan.save('gan_model.h5')